<h1>Buscando dados da internet</h1>

<h3>Laboratório de Coletas, Preparação e Análise de Dados.</h3>
Desenvolvendo um crawler básico.

Prof. Luan Garcia

Baseado no material de Richard Mitchel
https://github.com/REMitchell/python-scraping

Com a biblioteca Beautiful Soup vimos como é possível tratar documentos HTML como objetos no modelo DOM, buscando informação pelas *tags* e seus atributos. Porém, para chegarmos ao ponto de processar um documento HTML precisamos realizar o download do documento a partir do seu servidor web e como descobrir novos documentos a partir de um link de origem.

Neste notebook, veremos como é possível fazer requisições HTTP utilizando a biblioteca urllib e como lidar com alguns problemas básicos que podem acontecer quando estamos trabalhando com um crawler.


Primeiro passo é importar o módulo request da biblioteca urllib. Não é necessário instalar, pois é uma biblioteca nativa do Python.

In [ ]:
from urllib.request import urlopen


Para fazer o download do documento web, basta sabermos o endereço do documento e chamar o método read() do **urlopen**.

In [ ]:
html = urlopen('http://127.0.0.1:8000/places/default/index')
print(html.read())

Para podermos tratar o documento utilizando a biblioteca BeautifulSoup podemos passar diretamente o documento carregado através da requisição HTTP e depois manipular conforme o necessário.

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://127.0.0.1:8000/places/default/index')
bs = BeautifulSoup(html.read(), 'html.parser')
print(bs.h1)

Um problema bastante comum é o site que estamos tentando acessar não estar disponível, seja porque erramos o endereço, seja porque o servidor está fora do ar.

Um solução simples é garantir que tenhamos um documento para tratar utilizando exceções.

In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

try:
    html = urlopen("http://127.0.0.1:8000/places/default/index")
except HTTPError as e:
    print("The server returned an HTTP error")
except URLError as e:
    print("The server could not be found!")
else:
    print(html.read())

O objetivo de um crawler é descobrir, a partir de um link raiz, outros documentos que estejam linkados de alguma forma.

Uma maneira simples de realizar isto é filtrando pelas tags de links de um documento HTML. Com isto, encontramos todos os links dentro de um documento.

Abaixo, um exemplo utilizando como raiz a página do ator Kevin Bacon na Wikipedia.

In [ ]:
html = urlopen('http://127.0.0.1:8000/places/default/index')
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find_all('a', id="places_neighbours__row"):
    if 'href' in link.attrs:
        print(link.attrs['href'])

In [ ]:
import re
html = urlopen('http://127.0.0.1:8000/places/default/index')
bs = BeautifulSoup(html, 'html.parser')
bs.find_all("td")

Evidentemente, nem todos links serão de nosso interesse. Podemos filtrar apenas os links que nos interessam procurando por algum padrão no endereço e utilizar uma expressão regular para realizar o filtro.

Abaixo, vamos filtrar apenas links para outros artigos da wiki, ignorando âncoras, links para arquivos, etc. Faremos isso nos aproveitando de conhecimento de como um verbete na wiki é organizado. Todos links de artigos estarão sempre dentro da tag **div** que contém um atributo de **id** com valor **'bodyContent'**. Além disso, todo link de verbete necessariamente começa com o endereço "/wiki/" e não possui ":" no endereço.

In [ ]:
import re
from bs4 import BeautifulSoup

html = urlopen('http://127.0.0.1:8000/places/default/view/Afghanistan-1')
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find('tr', {'id':'places_neighbours__row'}).find_all(
     href=re.compile('/places/default/view/')):
    print(link.attrs['href'])

Podemos generalizar este código em forma de uma função **getLinks()**. Isto possibilitará que busquemos os links de qualquer verbete da wiki.

In [ ]:
import re

def getLinks(articleUrl):
    html = urlopen('http://127.0.0.1:8000/places/default/index{}'.format(articleUrl))
    bs = BeautifulSoup(html, 'html.parser')
    return bs.find_all(href=re.compile("/places/default/view/"))
for link in bs.find_all(href=re.compile("/places/default/index/")):
    links = getLinks('/default/index/')
    for i in range(len(links)):
        newArticle = links[i].attrs['href']
        print(newArticle)


In [ ]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

def getLinks(articleUrl):
    html = urlopen('http://127.0.0.1:8000/places/default/index{}'.format(articleUrl))
    bs = BeautifulSoup(html, 'html.parser')
    links = bs.find_all(href=re.compile("/places/default/index/"))
    for link in links:
        newArticle = link.attrs['href']
        print(newArticle)
        getLinks(newArticle)  # Recursive call

getLinks('/default/index')

A função **getLinks()** anterior funciona se quisermos encontrar todos os links de uma única página, porém, se quisermos fazer um crawler efetivo, precisamos procurar por páginas linkadas dentro de outras páginas de forma recursiva. Podemos fazer isso chamando a nosa própria função de procurar link de forma recursiva.

In [ ]:
pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen('http://127.0.0.1:8000/places/default/view/Afghanistan-1{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_all(href=re.compile("/places/default/iso")):
        if link.attrs['href'] not in pages:
            newPage = link.attrs['href']
            print(newPage)
            pages.add(newPage)
            getLinks(newPage)
            
            


In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

link_list = []

def busca_links(site):
      soup = BeautifulSoup(site.read(), 'html.parser')
      table=soup.find_all('td')
      
      for tag in table:
            tag_link = tag.next_element.next_element
            link_list.append(tag_link.attrs['href'])

      for tag in soup.find_all('a'):
            if tag.string == "Next >":
                  if 'href' in tag.attrs:
                        html = urlopen('http://127.0.0.1:8000' + tag.attrs['href'])
                        busca_links(html)


html = urlopen('http://127.0.0.1:8000/places/default/index')

busca_links(html)

print(link_list)

Experimente executar o comando a seguir e veja o que acontece. 

In [ ]:
getLinks('')

O que aconteceu? Será que isto é um problema? Se for, como podemos solucioná-lo?

In [ ]:
from urllib.request import urlopen

In [12]:
import requests
from bs4 import BeautifulSoup

url = "https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250"

# Adicionando um cabeçalho de usuário
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Verifica se houve erro na solicitação
except requests.HTTPError as e:
    print("The server returned an HTTP error:", e)
except requests.RequestException as e:
    print("Request error:", e)
else:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
lista_series = soup.find('ul', class_='ipc-metadata-list ipc-metadata-list--dividers-between sc-a1e81754-0 eBRbsI compact-list-view ipc-metadata-list--base')
if lista_series:
    print("Elemento <ul> encontrado.")
    
    series = lista_series.find_all('li')
    num_series = len(series)
    print(f"Encontradas {num_series} séries dentro de <ul>.")
    
    if series:
        print("Elementos <li> encontrados dentro de <ul>.")
        for serie in series:
            print(serie.prettify())  # Exibe o conteúdo de cada <li>
    else:
        print("Nenhum elemento <li> encontrado dentro de <ul>.")
else:
    print("Elemento <ul> não encontrado.")




Elemento <ul> encontrado.
Encontradas 250 séries dentro de <ul>.
Elementos <li> encontrados dentro de <ul>.
<li class="ipc-metadata-list-summary-item sc-10233bc-0 iherUv cli-parent">
 <div class="sc-e5a25b0f-0 jQjDIb cli-poster-container">
  <div class="ipc-poster ipc-poster--base ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2" role="group">
   <div aria-label="add to watchlist" class="ipc-watchlist-ribbon ipc-focusable ipc-watchlist-ribbon--s ipc-watchlist-ribbon--base ipc-watchlist-ribbon--loading ipc-watchlist-ribbon--onImage ipc-poster__watchlist-ribbon" role="button" tabindex="0">
    <svg class="ipc-watchlist-ribbon__bg" height="34px" role="presentation" viewbox="0 0 24 34" width="24px" xmlns="http://www.w3.org/2000/svg">
     <polygon class="ipc-watchlist-ribbon__bg-ribbon" fill="#000000" points="24 0 0 0 0 32 12.2436611 26.2926049 24 31.7728343">
     </polygon>
     <polygon class="ipc-watchlist-ribbon__bg-hover" points="24 0 0 0 0 32 12.2436611 26.29260

In [49]:
import requests
from bs4 import BeautifulSoup

def pega_autores():
    # Adicionando um cabeçalho de usuário
    url = "https://www.imdb.com/title/tt0903747/?ref_=chttvtp_t_1"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    try:
        response_serie = requests.get(url, headers=headers)  # Corrigindo a passagem da URL
        response_serie.raise_for_status()
    except requests.HTTPError as e:
        print("The server returned an HTTP error:", e)
        return None
    except requests.RequestException as e:
        print("Request error:", e)
        return None
    else:
        html_serie = response_serie.text
        soup = BeautifulSoup(html_serie, 'html.parser')
        lista_div = soup.find('div', class_="ipc-chip-list__scroller")
        lista_span = lista_div.find_all('span', class_="ipc-chip__text")
        for span in lista_span:
            print(span.text)
        

# Exemplo de uso:
pega_autores()




            

Crime
Drama
Thriller


In [45]:
import requests
from bs4 import BeautifulSoup
import re   

url = "https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250"

# Adicionando um cabeçalho de usuário
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
except requests.HTTPError as e:
    print("The server returned an HTTP error:", e)
except requests.RequestException as e:
    print("Request error:", e)
else:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    lista_series = soup.find('ul', class_="ipc-metadata-list ipc-metadata-list--dividers-between sc-a1e81754-0 eBRbsI compact-list-view ipc-metadata-list--base")
    if lista_series:
        series = lista_series.find_all('li')
        
    for serie in series:
        titulo_elemento = serie.find(class_= "ipc-title__text").text
        print(titulo_elemento)
        ano_estreia_elemento = serie.find(class_="sc-b0691f29-8 ilsLEX cli-title-metadata-item").text
        div = soup.find('div', class_="sc-b0691f29-7 hrgukm cli-title-metadata")
        spans = div.find_all('span', class_="sc-b0691f29-8 ilsLEX cli-title-metadata-item")
        
        if len(spans)>= 2:
            numero_episodios = spans[1].text
            print(numero_episodios)
            
        rating = serie.find(class_= "ipc-rating-star").text
        link_pagina_filme = serie.find("a", class_= "ipc-title-link-wrapper")
        re2_result = re.search(r'\d{4}',ano_estreia_elemento)
        
        if re2_result:
            ano_estreia_elemento = re.search(r'\d{4}',ano_estreia_elemento).group(0)
            print(ano_estreia_elemento)
            
        re_result = re.search(r'^\d\.\d',rating) #aplicando a regex para extrair apenas o padrão 'digito ponto digito'
        
        if re_result:
            rating = re.search(r'^\d\.\d',rating).group(0) #string resultante da regex
            print(rating)
            
        link = link_pagina_filme["href"]
        print(link)





1. Breaking Bad
62 eps
2008
9.5
/title/tt0903747/?ref_=chttvtp_t_1
2. Planeta Terra II
62 eps
2016
9.5
/title/tt5491994/?ref_=chttvtp_t_2
3. Planeta Terra
62 eps
2006
9.4
/title/tt0795176/?ref_=chttvtp_t_3
4. Irmãos de Guerra
62 eps
2001
9.4
/title/tt0185906/?ref_=chttvtp_t_4
5. Chernobyl
62 eps
2019
9.3
/title/tt7366338/?ref_=chttvtp_t_5
6. A Escuta
62 eps
2002
9.3
/title/tt0306414/?ref_=chttvtp_t_6
7. Avatar: A Lenda de Aang
62 eps
2005
9.3
/title/tt0417299/?ref_=chttvtp_t_7
8. Planeta Azul II
62 eps
2017
9.3
/title/tt6769208/?ref_=chttvtp_t_8
9. Família Soprano
62 eps
1999
9.2
/title/tt0141842/?ref_=chttvtp_t_9
10. Cosmos: Uma Odisseia do Espaço-Tempo
62 eps
2014
9.3
/title/tt2395695/?ref_=chttvtp_t_10
11. Cosmos
62 eps
1980
9.3
/title/tt0081846/?ref_=chttvtp_t_11
12. Nosso Planeta
62 eps
2019
9.3
/title/tt9253866/?ref_=chttvtp_t_12
13. Game of Thrones
62 eps
2011
9.2
/title/tt0944947/?ref_=chttvtp_t_13
14. O Mundo em Guerra
62 eps
1973
9.2
/title/tt0071075/?ref_=chttvtp_t_14
15. Bl